In [7]:
%pip install pandas

     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.9 kB 640.0 kB/s eta 0:00:01
     -------------------------------------- 60.9/60.9 kB 803.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.0 MB 10.2 MB/s eta 0:00:02
   -------- ------------------------------- 2.4/11.0 MB 21.8 MB/s eta 0:00:01
   ------------ --------------------------- 3.5/11.0 MB 22.1 MB/s eta 0:00:01
   ---------------- ----------------------- 4.5/11.0 MB 23.9 MB/s eta 0:00:01
   --------------------- ------------------ 5.9/11.0 MB 25.2 MB/s eta 0:00:01
   ------------------------ --------------- 6.6/11.0 MB 24.9 MB/s eta 0:00:01
   ---------------------------- ----------- 7.9/11.0 MB 25.3 MB/s eta 0:00:01
   --------------------------------- ------ 9.2/11.0 MB 25.5 MB/s eta 0:00:01
   -------------------------------------- - 10.5/11.0 MB 27.3 MB/s eta 0:0


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
from datetime import datetime
import os

transactions_df = pd.read_csv(r"C:\Users\kusha\OneDrive\Desktop\Projects\DevDolphins\Blob files\Chunks\chunk_1_20250624_202535.csv")
customer_df = pd.read_csv(r"C:\Users\kusha\OneDrive\Desktop\Projects\DevDolphins\Blob files\customer data\customer data.csv")
transactions_df.head()
customer_df.head()

In [20]:
transactions_df.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55,0
1,0,'C352968107','2','M','28007','M348934600','28007','es_transportation',39.68,0
2,0,'C2054744914','4','F','28007','M1823072687','28007','es_transportation',26.89,0
3,0,'C1760612790','3','M','28007','M348934600','28007','es_transportation',17.25,0
4,0,'C757503768','5','M','28007','M348934600','28007','es_transportation',35.72,0


In [ ]:
customer_df.head()

,Source,Target,Weight,typeTrans,fraud
0,'C1093826151','M348934600',4.55,'es_transportation',0
1,'C352968107','M348934600',39.68,'es_transportation',0
2,'C2054744914','M1823072687',26.89,'es_transportation',0
3,'C1760612790','M348934600',17.25,'es_transportation',0
4,'C757503768','M348934600',35.72,'es_transportation',0


In [ ]:
merged_df = transactions_df.merge(
    customer_df,
    how='inner',
    left_on=['customer', 'merchant', 'category','amount'],
    right_on=['Source', 'Target', 'typeTrans','weight']
)
merged_df

AttributeError: 'DataFrame' object has no attribute 'select'